In [1]:
from data_transform import *
from train_predict_submit import *


In [2]:
data_train = pd.read_csv("./data/train_sample.csv", encoding='utf-8', engine='python', sep=',')
sub_test = pd.read_csv("./data/test_sample.csv")

In [3]:
data_train.shape

(1034, 16)

In [4]:

data_clean = data_train.copy()
nulls = data_clean['building_id'].isnull()
data_clean = data_clean[~nulls].reset_index(drop=True)
data_clean['number_of_photos'] = data_clean.photos.apply(split_to_len)
data_clean['bedrooms'] = munge_bedbath(data_clean.bedrooms)
data_clean['bathrooms'] = munge_bedbath(data_clean.bathrooms)
data_clean['price'] = scale_variable(data_clean['price'])

data_clean = conv_geo_tocx(data_clean)

> /Users/EH/projects/kaggle/kaggle_rental_listing/data_transform.py(87)munge_bedbath()
-> col = col.fillna(-1)
(Pdb) col.fillna(-1)
0       3
1       2
2       1
3       1
4       4
5       4
6       2
7       1
8       1
9       4
10      0
11      1
12      2
13      2
14      1
15      0
16      2
17      1
18      4
19      1
20      3
21      2
22      1
23      1
24      4
25      4
26      1
27      1
28      1
29      2
       ..
977     0
978     2
979     2
980     2
981     1
982     4
983     3
984     2
985     2
986     2
987     1
988     1
989     0
990     3
991     0
992     0
993     2
994     2
995     0
996     1
997     1
998     2
999     2
1000    3
1001    2
1002    1
1003    0
1004    0
1005    1
1006    3
Name: bedrooms, dtype: object
(Pdb) col.fillna(-1).shape
(1007,)
(Pdb) n
> /Users/EH/projects/kaggle/kaggle_rental_listing/data_transform.py(88)munge_bedbath()
-> col = col.apply(tointfloat)
(Pdb) n
> /Users/EH/projects/kaggle/kaggle_rental_listing/data_tran

ValueError: Can only tuple-index with a MultiIndex

In [5]:

    
il_fix = []
iremove = []
for i, d in enumerate(data_clean.interest_level):
    if len(str(d)) > 6:
        if data_clean.building_id.loc[i] in ['medium', 'high', 'low']:
            il_fix += [i]
        else: 
            iremove += [i]
    elif str(d).lower() in ['nan', '']:
        iremove += [i]
        
data_clean.drop(iremove, inplace=True)
data_clean.interest_level.loc[il_fix] = data_clean.building_id.loc[il_fix]
data_clean.drop(data_clean[~data_clean.interest_level.isin(['medium', 'high', 'low'])].index, inplace=True)
data_clean['interest_level'], _classes = encode_target(data_clean.interest_level)
data_clean = data_clean[~data_clean.price.isnull()][~data_clean.longitude.isnull()]

exclude_columns = [
   'building_id', 'listing_id', 'photos', 'Unnamed: 0', 'latitude', u'description',u'longitude', u'display_address', u'manager_id', 
    u'street_address', u'created']


for ec in exclude_columns:
    try:
        del data_clean[ec]
    except:
        print('not deleted %s ' % ec)
        continue

# #apply the function to the original df:
# outlier = is_outlier(data_clean.price)
# #filter to only non-outliers:
# data_clean = data_clean[~(outlier)]



/Users/EH/anaconda/lib/python3.6/site-packages/pandas/core/indexing.py:141: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/Users/EH/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:18: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


In [6]:


y = np.array(data_clean.interest_level)
X = np.array(data_clean.drop('interest_level', axis=1))
Xtrain, ytrain, Xtest, ytest = split_test_train(X, y)
Xtrain.shape, ytrain.shape, Xtest.shape, ytest.shape

((794, 5), (794,), (199, 5), (199,))

In [7]:

clfs = tune_model(Xtrain, ytrain)

classifier = train_model(Xtrain, ytrain, clfs)

pred = classifier.predict(Xtest)

print(pd.DataFrame(confusion_matrix(ytest, pred, labels=[0,1,2])))
print(classification_report(y_true=ytest, y_pred=pred, target_names=_classes))



ValueError: could not convert string to float: "['Dishwasher', 'Hardwood Floors', 'No Fee']"